## Цель разработки
Разработать интеллектуального Telegram-бота для анализа фондового рынка, который предоставляет пользователям прогнозы цен акций и торговые рекомендации на основе машинного обучения

## Основной функционал
- **Анализ акций** - загрузка и обработка исторических данных с Yahoo Finance
- **Прогнозирование** - использование 5 различных алгоритмов (Random Forest, Ridge, ARIMA, ETS, LSTM)
- **Визуализация** - построение графиков с историческими данными и прогнозом
- **Торговые рекомендации** - выявление оптимальных точек для покупки/продажи
- **Расчёт прибыли** - симуляция инвестиционной стратегии

## Особенности
- Автоматический выбор лучшей модели по метрике MAPE
- Интуитивный интерфейс с инлайн-кнопками
- Логирование всех запросов для анализа эффективности

## Технические требования
- Токен Telegram Bot от @BotFather

# Использование бота
Вызовите команду /help, там всё описано

In [1]:
!pip install python-telegram-bot

In [2]:
import logging
import warnings
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import asyncio
import csv
import os
import io
import yfinance as yf
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from telegram import Update, InlineKeyboardButton, InlineKeyboardMarkup, BotCommand
from telegram.ext import Application, CommandHandler, MessageHandler, filters, CallbackContext, CallbackQueryHandler
import nest_asyncio
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [3]:
nest_asyncio.apply() # Без этого будет ошибка Event loop is already running

# Делаем логирование
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO)
logger = logging.getLogger(__name__)

In [4]:
# Нейросетевая модель LSTM
class LSTM(nn.Module):
    def __init__(self, input_size=1, hidden_size=50, num_layers=2, output_size=1, dropout=0.2):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(hidden_size, 25)
        self.fc2 = nn.Linear(25, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.lstm(x, (h0, c0))
        out = self.dropout(out[:, -1, :])
        out = torch.relu(self.fc1(out))
        return self.fc2(out)

In [5]:
class TelegramBot:
    def __init__(self):
        self.user_sessions = {}
        self.tickers = ["AAPL - Apple", "MSFT - Microsoft", "GOOGL - Google", "AMZN - Amazon", "TSLA - Tesla"]

    # Начальная ф-я взаимодействия с ботом и начало нового анализа
    async def start(self, update, context):
        user_id = update.effective_user.id
        self.user_sessions[user_id] = {'step': 'ticker'}
        await update.message.reply_text('Введи тикер компании (например AAPL, AMZN)')

    # Показывает список популярных тикеров
    async def list_tickers(self, update, context):
        text = "Популярные тикеры:\n" + "\n".join(self.tickers)
        await update.message.reply_text(text)

    # Показывает справку по использованию бота
    async def help_command(self, update, context):
        text = "Как использовать:\n/start - начать анализ\n/list - список тикеров\n"
        await update.message.reply_text(text)

    # Обработчик ввода тикера
    async def handle_text(self, update, context):
        user_id = update.effective_user.id
        text = update.message.text.strip()
        if user_id not in self.user_sessions or self.user_sessions[user_id].get('done'):
            self.user_sessions[user_id] = {'step': 'ticker'}
            session = self.user_sessions[user_id]
            ticker = text.upper()
            if not await self.check_ticker(ticker):
                await update.message.reply_text("Не нашел такой тикер. Введи команду /list и выбери понравившийся")
                return

            session['ticker'] = ticker
            session['step'] = 'amount'
            await update.message.reply_text(f"Смотрим {ticker}. Теперь введи сумму для анализа")
            return

        session = self.user_sessions[user_id]

        if session['step'] == 'ticker':
            ticker = text.upper()
            if not await self.check_ticker(ticker):
                await update.message.reply_text("Не нашел такой тикер. Введи команду /list и выбери понравившийся")
                return

            session['ticker'] = ticker
            session['step'] = 'amount'
            await update.message.reply_text(f"Смотрим {ticker}. Теперь введи сумму для анализа")

        elif session['step'] == 'amount':
            try:
                amount = float(text)
                if amount <= 0:
                    raise ValueError
            except:
                await update.message.reply_text(f"Сумма должна быть числом и больше нуля. Ты же ввёл {text}")
                return

            session['amount'] = amount
            await self.do_analysis(update, user_id)

    # Проверка валидности тикера
    async def check_ticker(self, ticker):
        try:
            stock = yf.Ticker(ticker)
            price = stock.info.get('regularMarketPrice')
            return price is not None and price > 0
        except:
            return False

    # Загрузка данных с Yahoo Finance за последние 2 года
    async def get_stock_data(self, ticker):
        end = datetime.now()
        start = end - timedelta(days=730)
        data = yf.Ticker(ticker).history(start=start, end=end)
        return data

    # Подготовка признаков для моделей
    def prepare_data(self, data):
        df = data[['Close']].copy()
        df['Returns'] = df['Close'].pct_change()
        df['MA_7'] = df['Close'].rolling(7).mean()
        df['MA_30'] = df['Close'].rolling(30).mean()
        return df.dropna()

    # Создание временных лагов
    def make_lags(self, series, n_lags=30):
        df = pd.DataFrame(series, columns=['Close'])
        for i in range(1, n_lags + 1):
            df[f'lag_{i}'] = series.shift(i)
        return df.dropna()

    # Далее идут ф-и обучения моделей
    def train_ml_models(self, X_train, y_train, X_test, y_test):
        rf = RandomForestRegressor(n_estimators=50, max_depth=15, min_samples_split=10, random_state=42)
        rf.fit(X_train, y_train)
        rf_pred = rf.predict(X_test)
        rf_mape = mean_absolute_percentage_error(y_test, rf_pred)

        ridge = Ridge(alpha=1.0)
        ridge.fit(X_train, y_train)
        ridge_pred = ridge.predict(X_test)
        ridge_mape = mean_absolute_percentage_error(y_test, ridge_pred)

        return {
            'RandomForest': {'model': rf, 'mape': rf_mape},
            'Ridge': {'model': ridge, 'mape': ridge_mape}
        }

    def train_arima(self, train_data, test_data):
        model = ARIMA(train_data, order=(5,1,0))
        fitted = model.fit()
        forecast = fitted.forecast(steps=len(test_data))
        mape = mean_absolute_percentage_error(test_data, forecast)
        return {'model': fitted, 'mape': mape}

    def train_ets(self, train_data, test_data):
        model = ExponentialSmoothing(train_data, seasonal='add', seasonal_periods=30)
        fitted = model.fit()
        forecast = fitted.forecast(len(test_data))
        mape = mean_absolute_percentage_error(test_data, forecast)
        return {'model': fitted, 'mape': mape}

    def train_lstm(self, train_data, test_data, lookback=30, epochs=150):
        scaler = StandardScaler()
        train_scaled = scaler.fit_transform(train_data.values.reshape(-1, 1))
        test_scaled = scaler.transform(test_data.values.reshape(-1, 1))

        def create_sequences(data, lookback):
            X, y = [], []
            for i in range(lookback, len(data)):
                X.append(data[i-lookback:i, 0])
                y.append(data[i, 0])
            return np.array(X), np.array(y)

        X_train, y_train = create_sequences(train_scaled, lookback)
        X_test, y_test = create_sequences(test_scaled, lookback)

        if len(X_train) == 0 or len(X_test) == 0:
            return {'model': None, 'mape': 999}

        X_train_t = torch.FloatTensor(X_train).unsqueeze(-1)
        y_train_t = torch.FloatTensor(y_train).unsqueeze(-1)
        X_test_t = torch.FloatTensor(X_test).unsqueeze(-1)
        y_test_t = torch.FloatTensor(y_test).unsqueeze(-1)

        model = LSTM(input_size=1, hidden_size=50, num_layers=2, output_size=1)
        criterion = nn.MSELoss()
        optimizer = optim.Adam(model.parameters(), lr=0.0001)

        model.train()
        for epoch in range(epochs):
            for i in range(0, len(X_train_t), 64):
                batch_x = X_train_t[i:i+64]
                batch_y = y_train_t[i:i+64]
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            test_pred = model(X_test_t)
            test_pred = test_pred.numpy()

        test_pred = scaler.inverse_transform(test_pred)
        y_test_orig = scaler.inverse_transform(y_test.reshape(-1, 1))
        mape = mean_absolute_percentage_error(y_test_orig.flatten(), test_pred.flatten())

        return {
            'model': model,
            'scaler': scaler,
            'mape': mape,
            'lookback': lookback
        }

    # Прогнозирование с помощью LSTM модели
    def forecast_lstm(self, model_info, last_data, steps=30):
        model = model_info['model']
        scaler = model_info['scaler']
        lookback = model_info['lookback']

        last_scaled = scaler.transform(last_data.reshape(-1, 1))
        forecasts = []
        current_seq = torch.FloatTensor(last_scaled[-lookback:]).unsqueeze(0)

        model.eval()
        with torch.no_grad():
            for _ in range(steps):
                pred_scaled = model(current_seq)
                pred = scaler.inverse_transform(pred_scaled.numpy())[0, 0]
                forecasts.append(pred)

                new_val = pred_scaled.numpy()[0, 0]
                new_seq = np.append(current_seq.numpy()[0, 1:, 0], new_val)
                current_seq = torch.FloatTensor(new_seq).unsqueeze(0).unsqueeze(-1)

        return np.array(forecasts)

    # Поиск локальных минимумов и максимумов
    def find_extremes(self, prices, window=5):
        mins = []
        maxs = []
        for i in range(window, len(prices) - window):
            if all(prices[i] <= prices[i-j] for j in range(1, window+1)) and all(prices[i] <= prices[i+j] for j in range(1, window+1)):
                mins.append(i)
            elif all(prices[i] >= prices[i-j] for j in range(1, window+1)) and all(prices[i] >= prices[i+j] for j in range(1, window+1)):
                maxs.append(i)
        return mins, maxs

    # Расчёт потенциальной прибыли на основе рекомендаций
    def calc_profit(self, prices, mins, maxs, money):
        if not mins or not maxs:
            return 0, []

        cash = money
        shares = 0
        transactions = []
        min_i = 0
        max_i = 0

        day = 0
        while day < len(prices) and min_i < len(mins) and max_i < len(maxs):
            if min_i < len(mins) and mins[min_i] == day and cash > 0:
                price = prices[day]
                shares = cash / price
                cash = 0
                transactions.append(('BUY', day, price, shares))
                min_i += 1
            elif max_i < len(maxs) and maxs[max_i] == day and shares > 0:
                price = prices[day]
                cash = shares * price
                shares = 0
                transactions.append(('SELL', day, price, cash))
                max_i += 1
            day += 1

        if shares > 0:
            cash = shares * prices[-1]
            transactions.append(('SELL', len(prices)-1, prices[-1], cash))

        profit = cash - money
        return profit, transactions

    # Построение графика
    def make_plot(self, history, forecast, ticker):
        plt.figure(figsize=(12, 6))
        plt.plot(history.index, history['Close'], 'b-', label='История', linewidth=1)

        last_date = history.index[-1]
        future_dates = [last_date + timedelta(days=i) for i in range(1, len(forecast)+1)]
        plt.plot(future_dates, forecast, 'r--', label='Прогноз', linewidth=2)

        plt.title(f'Цены акций {ticker}')
        plt.xlabel('Дата')
        plt.ylabel('Цена ($)')
        plt.legend()
        plt.grid(True, alpha=0.3)
        plt.xticks(rotation=45)
        plt.tight_layout()

        buf = io.BytesIO()
        plt.savefig(buf, format='png', dpi=150)
        buf.seek(0)
        plt.close()
        return buf

    # Логирование пользовательских запросов
    async def log_request(self, user_id, ticker, amount, best_model, mape, profit):
        csv_file = 'logs.csv'
        file_exists = os.path.isfile(csv_file)

        with open(csv_file, 'a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            if not file_exists:
                writer.writerow(['time', 'user_id', 'ticker', 'amount', 'model', 'mape', 'profit'])
            writer.writerow([datetime.now(), user_id, ticker, amount, best_model, mape, profit])

    # Финальная ф-я
    async def do_analysis(self, update, user_id):
        try:
            session = self.user_sessions[user_id]
            ticker = session['ticker']
            amount = session['amount']

            await update.message.reply_text("Скачиваю данные")
            data = await self.get_stock_data(ticker)

            if data.empty:
                await update.message.reply_text("Не получилось загрузить данные")
                return

            df = self.prepare_data(data)
            prices = df['Close'].values

            split = int(len(prices) * 0.8)
            train = prices[:split]
            test = prices[split:]

            await update.message.reply_text("Обучаю модели")

            models = {}
            features_data = None

            # ML модели
            lag_data = self.make_lags(pd.Series(prices))
            X = lag_data.drop('Close', axis=1)
            y = lag_data['Close']
            split_lag = int(len(X) * 0.8)
            X_train, X_test = X[:split_lag], X[split_lag:]
            y_train, y_test = y[:split_lag], y[split_lag:]
            ml_models = self.train_ml_models(X_train, y_train, X_test, y_test)
            models.update(ml_models)
            features_data = X

            # ARIMA
            arima_res = self.train_arima(train, test)
            models['ARIMA'] = arima_res

            # ETS
            ets_res = self.train_ets(train, test)
            models['ETS'] = ets_res

            # LSTM
            lstm_res = self.train_lstm(pd.Series(train), pd.Series(test))
            models['LSTM'] = lstm_res

            best_name = None
            best_mape = 999

            for name, res in models.items():
                if res['mape'] < best_mape and res['model'] is not None:
                    best_mape = res['mape']
                    best_name = name
                    best_model = res

            if best_name is None:
                await update.message.reply_text("Не вышло обучить модели")
                return

            await update.message.reply_text(f"Лучшая модель: {best_name}")

            await update.message.reply_text("Делаю прогноз")

            if best_name in ['RandomForest', 'Ridge']:
                last_feat = features_data.iloc[-1:].values
                forecast = []
                current = last_feat.copy()
                for _ in range(30):
                    pred = best_model['model'].predict(current)[0]
                    forecast.append(pred)
                    current = np.roll(current, -1)
                    current[0, -1] = pred
            elif best_name == 'ARIMA':
                forecast = best_model['model'].forecast(steps=30)
            elif best_name == 'ETS':
                forecast = best_model['model'].forecast(steps=30)
            elif best_name == 'LSTM':
                last_seq = prices[-best_model['lookback']:]
                forecast = self.forecast_lstm(best_model, last_seq, 30)

            forecast = np.array(forecast)

            current_price = prices[-1]
            future_price = forecast[-1]
            change_pct = ((future_price - current_price) / current_price) * 100

            all_prices = np.concatenate([prices[-30:], forecast])
            mins, maxs = self.find_extremes(all_prices)
            profit, transactions = self.calc_profit(all_prices, mins, maxs, amount)

            await update.message.reply_text("Рисую график")
            plot_buf = self.make_plot(data.tail(100), forecast, ticker)

            # Ф-я для лучшего пониманя, через сколько покупать/продавать
            def format_days(days):
                if not days:
                    return "нет"
                result = []
                for d in days:
                    if d == 1:
                        result.append("завтра")
                    elif d == 2:
                        result.append("послезавтра")
                    elif d % 10 == 1 and d != 11:
                        result.append(f"через {d} день")
                    elif d % 10 in [2,3,4] and d not in [12,13,14]:
                        result.append(f"через {d} дня")
                    else:
                        result.append(f"через {d} дней")
                return ", ".join(result)

            buy_days = [d-29 for d in mins if d >= 30]
            sell_days = [d-29 for d in maxs if d >= 30]

            report = f"""Отчёт по {ticker}:

                Текущая цена: ${float(round(current_price, 2))}
                Прогноз через 30 дней: ${float(round(future_price, 2))}
                Изменение: {float(round(change_pct, 2))}%

                Покупать: {format_days(buy_days)}
                Продавать: {format_days(sell_days)}

                Инвестиция: ${float(round(amount, 2))}
                Прибыль: ${float(round(profit, 2))}
                Доходность: {float(round((profit/amount)*100, 2))}%

                Модель: {best_name}
                Точность: {float(round(best_mape * 100, 2))}%
            """

            await update.message.reply_photo(photo=plot_buf, caption=report)

            await self.log_request(user_id, ticker, amount, best_name, best_mape, profit)

            await update.message.reply_text("Готово!", reply_markup=InlineKeyboardMarkup([
                    [InlineKeyboardButton("Новый анализ", callback_data="new_analysis"),
                    InlineKeyboardButton("Завершить", callback_data="end_session")]
                ]))

            self.user_sessions[user_id] = {'done': True}

        except Exception as e:
            await update.message.reply_text("Ошибка анализа. Попробуй другой тикер")

    async def handle_callback(self, update, context):
        query = update.callback_query
        await query.answer()

        if query.data == "new_analysis":
            user_id = query.from_user.id
            self.user_sessions[user_id] = {'step': 'ticker'}
            await query.edit_message_text("Введи тикер компании")

        elif query.data == "end_session":
            user_id = query.from_user.id
            if user_id in self.user_sessions:
                del self.user_sessions[user_id]
            await query.edit_message_text("Если захочешь снова - пиши тикер")

In [ ]:
# Запускает бота в Colab
async def run_bot():

    # Создание приложения
    application = Application.builder().token("Cюда нужно вставить токен бота").build()

    # Менюшка для телеги
    commands = [
        BotCommand("start", "Начать анализ акций"),
        BotCommand("list", "Список тикеров"),
        BotCommand("help", "Помощь по использованию"),
    ]

    await application.bot.set_my_commands(commands)

    # Создание бота
    bot = TelegramBot()

    # Регистрация обработчиков
    application.add_handler(CommandHandler("start", bot.start))
    application.add_handler(CommandHandler("list", bot.list_tickers))
    application.add_handler(CommandHandler("help", bot.help_command))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, bot.handle_text))
    application.add_handler(CallbackQueryHandler(bot.handle_callback))

    # Запуск бота
    await application.run_polling()

await run_bot()